In [2]:
import torch
from torchtext.data import Field, BucketIterator, TabularDataset, Iterator
from utils_transform import *
from transformer import *
from eval_lib import *
from torchnlp.metrics import get_moses_multi_bleu
from nltk.translate import bleu_score

In [3]:
train_path = "train_200k.csv" # TRAIN PATH MUST MATCH ORIGINAL
SOS, EOS, PAD, BOS = "<s>", "</s>", "<pad>", "<bos>" # Represents begining of context sentence
VOCAB_SIZE = 10000

In [4]:
bpemb_tr, bpemb_en = load_bpe(VOCAB_SIZE)

In [5]:
TR = Field(tokenize=bpemb_tr.encode, 
        lower=False, pad_token=PAD)
EN = Field(tokenize=bpemb_en.encode, 
    lower=False, pad_token=PAD, init_token=SOS, eos_token=EOS)
rev_tokenize_en = lambda tokenized: [EN.vocab.itos[i] for i in tokenized]
rev_tokenize_tr = lambda tokenized: [TR.vocab.itos[i] for i in tokenized]

print('reading in tabular dataset')
train, val, test = TabularDataset.splits(
  path='data/', 
  train=train_path,
  validation="val_10k.csv",
  test="test_10k.csv",
  format='tsv', 
  fields=[('src_context', TR), ('src', TR),
  ('trg_context', EN), ('trg', EN)])
print('done')

reading in tabular dataset
done


In [6]:
MIN_FREQ = 1
TR.build_vocab(train, min_freq=MIN_FREQ, max_size=VOCAB_SIZE)
EN.build_vocab(train, min_freq=MIN_FREQ, max_size=VOCAB_SIZE)

In [7]:
def load(path, tr_voc, en_voc):
    """Loads a trained model from memory for evaluation.
    """
    model = make_model(tr_voc, en_voc, N=6)
    model.load_state_dict(torch.load(path, map_location='cpu'))
    model.eval()
    return model

In [8]:
model = load('models/0214_baseline_bpe_200k_30.pt', len(TR.vocab), len(EN.vocab))

In [9]:
def log_likelihood(model, batch, pad_idx):
    """Calculates the log likelihood of a batch, given the model.
    """
    memory = model.encode(batch)
    total_prob = torch.zeros(batch.trg_y.size(0))
    for i in range(0, batch.trg_y.size(1)): # trg_len
        y_prev = batch.trg[:, :i + 1]
        out = model.decode(memory, batch.src_mask, 
                           y_prev.clone().detach(),
                           (subsequent_mask(y_prev.size(1))
                                    .type_as(batch.src.data)))
        probs = model.generator(out[:, -1]) # batch x vocab
        trg_index = batch.trg_y[:, i]
        prob_of_trg = probs.gather(1, trg_index.view(-1,1)) # not sure about this
        pad_mask = (trg_index != pad_idx).to(dtype=torch.float32)
        total_prob += (prob_of_trg.squeeze()) * pad_mask
    return total_prob

def eval_accuracy_helper(pad_idx, eval_iter, model):
  """ Helper function that calculates how well the model chooses the correct
  pronoun on the data in an iterator.
  """
  n_correct = 0.0
  n_total = 0.0
  for b in eval_iter:
    batch_correct, batch_incorrect = rebatch_for_eval(pad_idx, b)
    probs = torch.stack([
      log_likelihood(model, batch_correct, pad_idx), 
      log_likelihood(model, batch_incorrect, pad_idx)], dim=1) # n x 2
    correct = probs[:, 0] > probs[:, 1] # should assign higher probability to the left
    n_correct += torch.sum(correct).item()
    n_total += correct.size(0)
  print("Correct: %d / %d = %f" % (n_correct, n_total, (n_correct / n_total)))
  return probs

def eval_accuracy(pad_idx, path_to_test_set, model, TR, EN):
  """Calculates how well the model chooses the correct pronoun, given
  a dataset in TSV form. 
  """
  full_path = "data/%s" % (path_to_test_set)
  # Must be in order
  # !!! IMPT note there are five fields
  data_fields = [
  ('src_context', TR), ('src', TR),
  ('trg_context', EN), ('trg_correct', EN), ('trg_incorrect', EN)]
  print('Evaluating discriminative dataset: %s ' % (full_path))
  test = TabularDataset(
    full_path,
    format='tsv', 
    fields=data_fields)

  test_iter = Iterator(
    test, batch_size=100, sort_key=lambda x: 1, repeat=False, train=False)

  return eval_accuracy_helper(pad_idx, test_iter, model)


In [10]:
pad_idx = EN.vocab.stoi[PAD]

In [11]:
all_probs = eval_accuracy(pad_idx, "pro_stereotype.tsv", model, TR, EN)

Evaluating discriminative dataset: data/pro_stereotype.tsv 
Correct: 20 / 40 = 0.500000


In [12]:
all_probs = eval_accuracy(pad_idx, "female_subject.tsv", model, TR, EN)

Evaluating discriminative dataset: data/female_subject.tsv 
Correct: 0 / 40 = 0.000000


In [13]:
all_probs

tensor([[ -4.4412,  -3.9379],
        [-15.2876, -14.6235],
        [ -5.1715,  -4.0829],
        [ -4.7117,  -4.0060],
        [ -9.7297,  -8.8746],
        [-10.9707, -10.0292],
        [ -3.7112,  -3.0949],
        [ -4.2245,  -3.4824],
        [ -4.0854,  -3.5412],
        [ -5.4150,  -4.4954],
        [-11.1375, -10.4354],
        [ -4.3889,  -4.1192],
        [ -4.2272,  -3.5801],
        [ -8.8962,  -7.9357],
        [ -4.1279,  -3.2227],
        [ -5.2188,  -4.3702],
        [ -3.6341,  -3.3452],
        [ -3.8135,  -3.5356],
        [ -5.5106,  -4.3330],
        [-13.5022, -12.4666],
        [ -7.2734,  -6.8435],
        [ -5.0411,  -3.5484],
        [-14.8904, -14.0691],
        [ -4.9076,  -4.0623],
        [ -5.1163,  -4.2716],
        [-12.1053, -11.2125],
        [-14.5153, -12.7563],
        [ -4.2351,  -3.5396],
        [ -4.4835,  -3.2026],
        [ -6.3078,  -4.7284],
        [ -4.3690,  -3.3897],
        [ -7.4435,  -6.7757],
        [ -5.7988,  -4.3193],
        [ 

In [14]:
valid_iter = MyIterator(val, batch_size=512, device="cpu",
                        repeat=False, sort_key=lambda x: (len(x.src), len(x.trg), len(x.src_context)),
                        batch_size_fn=batch_size_fn, train=False)

In [24]:
def eval_bleu(pad_idx, eval_iter, model, max_len, start_symbol, end_symbol, rev_tokenize_trg, bpemb_en):
  """Calculates average BLEU score of a model on some validation iterator.
  """
  bleus = []
  for old_batch in eval_iter:
    batch = rebatch(pad_idx, old_batch)
    for i in range(batch.src.size(0)): # batch_size
      targets = batch.trg_y[i, :-1]
      targets_mask = (targets != pad_idx)
      targets = targets[targets_mask]
      trg_str = bpemb_en.decode(rev_tokenize_trg(targets))
        
      hypothesis = beam_decode(model, batch.src[i], batch.src_mask[i], batch.src_context[i],
       pad_idx, max_len, start_symbol, end_symbol, k=5)[1:-1] # cut off SOS, EOS
      hyp_str = bpemb_en.decode(rev_tokenize_trg(hypothesis)) 
      # bleu = get_moses_multi_bleu([hyp_str], [trg_str])
      bleu = bleu_score.sentence_bleu([trg_str], hyp_str)
      
      if bleu == None: # Error
        print(i, hyp_str, trg_str)
      else:
        print(bleu, hyp_str, trg_str, len(trg_str))
        bleus.append(bleu)
  return sum(bleus) / len(bleus)


In [25]:
eval_bleu(pad_idx, valid_iter, model, 10, EN.vocab.stoi[SOS],EN.vocab.stoi[EOS], rev_tokenize_en, bpemb_en)

1.0 harry. harry. 6
1.0 who? who? 4
1.0 what? what? 5
1.0 here. here. 5
1.0 jack. jack. 5
1.0 what? what? 5
1.0 here. here. 5


/Users/Susan/anaconda/envs/pytorch/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


1.2213386697554703e-77 no! no! 3
1.0 harry? harry? 6
1.0 what? what? 5
1.0 there. there. 6
1.0 what? what? 5
1.2213386697554703e-77 oh. oh. 3
1.0 what? what? 5
1.0 joe? joe? 4


/Users/Susan/anaconda/envs/pytorch/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


6.439988060393651e-155 no. none. 5
1.0 what? what? 5


...

0.10835142074863786 i think we should keep them here to keep i guess we need to keep 'em here to keep 'em safe, but i just worry that it's only gonna go from bad to worse. 110
0.011998492908040918 in the 0000s crisis time, when the in 0000, with the climbing budgets during the depression, the cold blooded warden thought he could keep his fortune on track by eliminating certain overhead costs. 163
0.022839444844700175 in fact, he was desperate for [narrator] in fact, george michael had developed a hopeless crush... on his ethics teacher, a woman named beth baerly. 118
0.06498419424680342 in september 0, 0000, crazy horse was killed on september 0, 0000, crazy horse, a man who fought his entire life to protect his people and won battle after battle against the united states army, is killed. 160
0.054663720015389106 the pegat bird must be very light frigates must be so lightweight that they can't afford the heavy oils that waterproof the plumage of other sea birds. 117
0.02126543229878098

0.20537361788556843 you know, when your father gets warned you know, when your father gets excited, police, priests, prayers, nothing is gonna stop him. 93
0.07702392199966192 ...what are the odds that i live what chance is there that mother and i have four kids and we live on a farm in cape may, new jersey? 100
0.25313649209113204 if i managed to find him before lieutenant kil but if i can reach him before lt. kilgore does, i think we'll have a chance. 76
0.012970166509482331 you better take your hands off 'cause i also know you better take your hands off me, because defending their president against an assault charge... will cost the montecito a hell of a lot more than i could ever win playing blackjack. 195
0.06948345122280154 when i was being held on that freighter when i was being held on that freighter and they had me handcuffed and lancer was sticking me with needles, there'one thing that kept me going. 143
0.04558898976324764 but gob felt that michael's kind [narrator] but gob 

0.007138896735252059 she's often spoken to poor, sc she's been talking with mostly poor local residents who've all been forced out to make room for the development, and they've been terrorized by taylor's thugs. 159
0.037760258230701116 i want you to sit here and sit with i'd like you to sit down with someone who's childlike enthusiasm and loyalty is gonna make him a terrific asset here at the sunshine center. 140
0.011792523601726751 if a man knows his last moments when a human knows his or her life is about to come to an end, it will tend to reach an anxiety level... that will cause it to emit highly noxious odors. 153
0.02995630933974062 we'll give them a couple of mar we'll wheel out a couple of marrakech rent boys, give them free tickets to disneyland in return for saying you begged them to piss on you. 138
0.027276861933107986 i don't know which one of them i don't know which made me happier, his smile up there in his wheelchair when he won the bronze medal or when he popped my c

0 _ inhisdebutact as a juggler, please welcome your favorite lobster boy-- and my precious son-- 92
0.004763726725225603 if that's true, if the religion well, if that's true, if religion does help people lead moral lives, then one should take very seriously the proposition that we're just going to eliminate it and let the devil take the hindmost. 195
0.005634730159805634 as billy the kid was growing up, a while billy the kid is now a part of the burgeoning cattle industry that's fueling america's growth... several hundred miles to tth, one man is doing everything he can to hold off western expansion. 198
0.07418062852628586 our question is to present the star of our the house is proud to present the star of our show, the late sick man and almost corpus delicti, back from a successful tour of the caribbean, 142
0.06526431866003739 when we first got married, i told my when we first got married and i told my friend fanny hogan the things we were doing, she told me to leave him and notify 

0.5013308784995479

In [114]:
get_moses_multi_bleu(["what is going"], ["what is going"])

0.0

In [17]:
bleu_score.sentence_bleu(["what is going"], "what is ")

0.5352614285189903

In [23]:
bleu_score.sentence_bleu(["jack."], "jack. ")

0.7598356856515925